## Object Detection - Tensorflow Models
- code reference taken from - https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/auto_examples/plot_object_detection_saved_model.html#sphx-glr-auto-examples-plot-object-detection-saved-model-py

In [ ]:
#import statements
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils



### Model Name List

In [ ]:
model_name_list = [
    #('centernet_hg104_1024x1024_coco17_tpu-32', '20200713'),
    #('centernet_hg104_512x512_coco17_tpu-8', '20200711'),
    #('centernet_resnet50_v1_fpn_512x512_coco17_tpu-8', '20200711'),
    #('centernet_resnet50_v2_512x512_coco17_tpu-8', '20200711'),
    #('efficientdet_d0_coco17_tpu-32', '20200711'),
    ('ssd_mobilenet_v2_320x320_coco17_tpu-8', '20200711'),
    #('faster_rcnn_resnet50_v1_640x640_coco17_tpu-8', '20200711'),
    #('mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8', '20200711')
    #('extremenet', '20200711') some error here, seems like extremenet does not have saved model dir in c/users/shubham/keras
]

### Fetch Images for inferences

In [ ]:
def download_images():
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/test_images/'
    filenames = ['image1.jpg', 'image2.jpg']
    image_paths = []
    for filename in filenames:
        image_path = tf.keras.utils.get_file(fname=filename,
                                            origin=base_url + filename,
                                            untar=False)
        image_path = pathlib.Path(image_path)
        image_paths.append(str(image_path))
    return image_paths

IMAGE_PATHS = download_images()

### Download the COCO Labels

In [ ]:
def download_labels(filename):
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
    label_dir = tf.keras.utils.get_file(fname=filename,
                                        origin=base_url + filename,
                                        untar=False)
    label_dir = pathlib.Path(label_dir)
    return str(label_dir)

LABEL_FILENAME = 'mscoco_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)

### Load label map data (for plotting)

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

### Download model from tensorflow.org/models
- More models can be found in the TensorFlow 2 Detection Model Zoo. To use a different model you will need the URL name of the specific model. This can be done as follows:
1. Right click on the Model name of the model you would like to use;
2. Click on Copy link address to copy the download link of the model;
3. Paste the link in a text editor of your choice. You should observe a link similar to download.tensorflow.org/models/object_detection/tf2/YYYYYYYY/XXXXXXXXX.tar.gz;
4. Copy the XXXXXXXXX part of the link and use it to replace the value of the MODEL_NAME variable in the code shown below;
5. Copy the YYYYYYYY part of the link and use it to replace the value of the MODEL_DATE variable in the code shown below.

In [ ]:
# Download and extract model
def download_model(model_name, model_date):
    base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(fname=model_name,
                                        origin=base_url + model_date + '/' + model_file,
                                        untar=True)
    return str(model_dir)

def download_models(model_name_list):
    model_dir_path_dict = {}
    for model_name in model_name_list:
        path = download_model(model_name[0], model_name[1])
        model_dir_path_dict[model_name[0]] = path
        print("downloaded {0}".format(model_name[0]))
    return model_dir_path_dict


#use below list to know model name and where it is downloaded locally
model_dir_path_dict = download_models(model_name_list)

### Load the downloaded models from directories

In [ ]:
def load_model(model_name, model_dir):
    print(model_dir)
    path_to_saved_model = model_dir + "/saved_model"

    print('Loading model...', end='')
    start_time = time.time()

    # Load saved model and build the detection function
    detect_fn = tf.saved_model.load(path_to_saved_model)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Done! Took {} seconds'.format(elapsed_time))
    return detect_fn



def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

def detect_objects(model):
    object_detections = []
    for image_path in IMAGE_PATHS:
        print('Running inference for {}... '.format(image_path), end='')

        image_np = load_image_into_numpy_array(image_path)

        # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
        input_tensor = tf.convert_to_tensor(image_np)
        # The model expects a batch of images, so add an axis with `tf.newaxis`.
        input_tensor = input_tensor[tf.newaxis, ...]

        # input_tensor = np.expand_dims(image_np, 0)
        detections = model(input_tensor)
        object_detections.append(detections)
    return object_detections

def visualise_results(detections):
    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    plt.show()

In [ ]:
def run_inference(model_name, model_dir):
    model = load_model(model_name, model_dir)
    detections = detect_objects(model)
    #process detections object to output list as per your need
    print("Model: {0}".format(model_name))
    visualise_results(detections)

In [ ]:
for model in model_dir_path_dict:
    run_inference(model, model_dir_path_dict[model])